## Working with an LLM programmatically

You have certainly interacted before with a Large Language Model (LLM) like ChatGPT. This is usually done through a UI or an application.

In this Notebook, we are going to use Python to connect and query an LLM directly through its API. For this Lab we have selected the model **Granite-3.1-8B-Instruct**.(https://huggingface.co/RedHatAI/granite-3.1-8b-instruct). This is a fully Open Source model (Apache 2.0 license) developed by IBM Research.

This model has already been deployed on the Lab cluster because even if it's a smaller model, it still needs a GPU with 24GB of RAM to run...

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages. We will then import the libraries we need.

In [ ]:
# Uncomment the following line only if you have not selected the right workbench image, or are using this notebook outside of the workshop environment.
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt
import json

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM API.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 512 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too "creative". After all, we're not trying to write a fancy poem here!

In [ ]:
# LLM Inference Server URL
inference_server_url = "http://granite-3-1-8b-instruct-predictor.ic-shared-llm.svc.cluster.local:8080"

# LLM definition
llm = ChatOpenAI(
    openai_api_key="EMPTY",   # Private model, we don't need a key
    openai_api_base=f"{inference_server_url}/v1",
    model_name="granite-3-1-8b-instruct",
    temperature=0.01,
    max_tokens=512,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    top_p=0.9,
    presence_penalty=0.5,
    model_kwargs={
        "stream_options": {"include_usage": True}
    }
)

We also need a **template** to be applied to every request we are sending to the model (the "Prompt").

When querying a model, you almost never want to send directly what the user has typed. On top of this entry, you need to give proper instructions to the model so that it knows how to handle it: what and how to answer, what NOT to answer, the tone it must use...

In [ ]:
template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """You are a helpful, respectful, and honest assistant.
        Answer each question clearly and concisely in a single response only.
        Do not continue the conversation or simulate dialogue unless explicitly asked.
        Never include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
        Ensure that your responses are socially unbiased and positive in nature.
        If a question does not make sense or is not factually coherent, explain why instead of trying to answer.
        If you don't know the answer to a question, say "I don't know".
        """),
    HumanMessagePromptTemplate.from_template("{input}"),
])

We are now ready to query the model!

In [ ]:
query = "What is Artificial Intelligence?"
prompt = template.invoke({"input": query})
response = llm.invoke(input=prompt)

Some information, like the tokens that were consumed and generated are present in the metadata. That can be useful to measure the consumption of the model.

In [ ]:
print(json.dumps(response.usage_metadata, indent=2))

You can come back to this notebook at section 3.7 for some optional exercises if you want.